In [1]:
import sys
import os
#sys.path.insert(0, "/usr/src/app/app")
def import_path(fullpath):
    """ 
    Import a file with full path specification. Allows one to
    import from anywhere, something __import__ does not do. 
    """
    path, filename = os.path.split(fullpath)
    filename, ext = os.path.splitext(filename)
    sys.path.append(path)
    module = __import__(filename)
   # reload(module) # Might be out of date
    del sys.path[-1]
    return module
import_path("/usr/src/app/app")

ValidationError: 1 validation error for Settings
AWS
  field required (type=value_error.missing)

In [ ]:
from PIL import Image

from app.pkg.ml.autotags.autotag import AutoTagger
from app.pkg.ml.buffer_converters import BytesConverter

In [ ]:
# user_images = {
#     "upper":["/usr/src/app/data/upper/b_bloose.png",
             
#              ]


# }

In [2]:
import torch
from transformers import AutoProcessor, CLIPModel, AutoTokenizer
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer =  AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
image_inputs = processor(images=image, return_tensors="pt")
        
image_features = model.get_image_features(**image_inputs)

In [ ]:
class LocalRecSys:
    """
    Recommends set of clothes (outfit)
    """
    def __init__(self, device="cuda:0"):
        self.model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
        self.processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.tokenizer =  AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
        self.device = device
        self.bytes_converter = BytesConverter()

    def forward(self,
                upper_clothes: List[Dict[str, io.BytesIO]],
                lower_clothes: List[Dict[str, io.BytesIO]],
                dresses_clothes: List[Dict[str, io.BytesIO]],
                user_photos:List[Dict[str, io.BytesIO]],
                prompt: str = None,
                top_n: int = 10,
               ) -> Dict[str, Dict[str, float]]:
        """
        Gets probability for each tag
        
        Args:
            upper_clothes: List[Dict[str, io.BytesIO]],
            lower_clothes: List[Dict[str, io.BytesIO]],
            dresses_clothes: List[Dict[str, io.BytesIO]],
            user_photos:List[Dict[str, io.BytesIO]] - photos of user to try_on in future.
                    Recommends to use one or less images
            prompt: str = None - extra prompt to search
        Returns:
            dict with sets of clothes
        
        Return example:
        [
            {
            "score":1,
            "clothes":[,]
            },
            {
            "score":0.93,
            "clothes":[,]
            },
        ]

        """
        pass
        # image = self.bytes_converter.bytes_to_image(input_data["image"])
        # tags = input_data['tags']
        # output_dict = self._get_tags(image, tags)
        # return output_dict

    @torch.inference_mode()
    def _get_image_embedding(self, images:List[Dict[str, Image]] ) -> Dict[str, float]:
        """
        Gets images embeddings
        
        Args:
            images:List[Dict[str, Image]] - images to get embeddings
            tags:Dict[str, List[str]]] - tags
            
        Returns:
            dict with probabilities   
        """
        image_inputs = self.processor(images=image, return_tensors="pt")
        self._input_to_device(image_inputs)

        image_features = self.model.get_image_features(**image_inputs)

        result_probabilities = {}
        for tag_group, text_tag_list in tags.items():
            result_probabilities[tag_group] = {}

            self.processor(text=text_tag_list, images=image, return_tensors="pt", padding=True)
            text_inputs = self.tokenizer(text_tag_list, padding=True, return_tensors="pt")
            self._input_to_device(text_inputs)

            text_features = self.model.get_text_features(**text_inputs)
            logits = image_features @ text_features.T
            probabilities = self.normalize(logits)
            probabilities = probabilities.flatten().tolist()

            for tag, prob in zip(text_tag_list, probabilities):
                r_prob = round(prob, 3)
                result_probabilities[tag_group][tag] = r_prob

        return result_probabilities

    def _input_to_device(self, input_data:dict):
        """
        Converts input dict values to device
        """
        for key, value in input_data.items():
            if isinstance(value, torch.Tensor):
                input_data[key] = value.to(self.device)
                